<a href="https://colab.research.google.com/github/Snap0dragon/Bangla-ANPR-using-Vision-Transformer/blob/main/Tr_OCR_Dataset_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Step 1: Install Kaggle and authenticate
!pip install kaggle

# Upload your Kaggle API key (kaggle.json) to the Colab environment
from google.colab import files
files.upload()  # Upload your kaggle.json file here




Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nayebhasin190021115","key":"59b499356696f0ee373a154aec71c5a7"}'}

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!mkdir "/content/drive/MyDrive/Colab Notebooks/Dataset"



In [5]:
!mv kaggle.json "/content/drive/MyDrive/Colab Notebooks/Dataset/"
!chmod 600 "/content/drive/MyDrive/Colab Notebooks/Dataset/kaggle.json"

In [6]:
dataset_url = "syednahinhossain/bangladeshi-license-plate-recognition-dataset"
!kaggle datasets download -d {dataset_url}

Dataset URL: https://www.kaggle.com/datasets/syednahinhossain/bangladeshi-license-plate-recognition-dataset
License(s): apache-2.0
 99% 1.69G/1.70G [00:14<00:00, 148MB/s]
100% 1.70G/1.70G [00:14<00:00, 123MB/s]


In [7]:
import zipfile
import os
dataset_zip = dataset_url.split("/")[-1] + ".zip"
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    all_files = zip_ref.namelist()
    # Filter only files within the "Character Recognition" folder
    char_rec_files = [f for f in all_files if f.startswith("Character Recognition/")]
    zip_ref.extractall(members=char_rec_files)

In [10]:
!mv "/content/Character Recognition" "/content/drive/MyDrive/Colab Notebooks/Dataset/"

In [13]:
# import os
# import xml.etree.ElementTree as ET
# import csv

# # Specify the folder containing the XML files
# input_folder = "/content/drive/MyDrive/Colab Notebooks/Dataset/Character Recognition/test/"
# output_csv = "/content/drive/MyDrive/Colab Notebooks/Dataset/output.csv"

# # Prepare the CSV file for writing
# with open(output_csv, mode="w", newline="", encoding="utf-8") as csv_file:
#     csv_writer = csv.writer(csv_file)
#     # Write the header row
#     csv_writer.writerow(["file_name", "txt"])

#     # Counter for renaming files
#     file_counter = 1

#     # Iterate through all files in the folder
#     for file_name in os.listdir(input_folder):
#         if file_name.endswith(".xml"):
#             # Parse the XML file
#             file_path = os.path.join(input_folder, file_name)
#             tree = ET.parse(file_path)
#             root = tree.getroot()

#             # Extract the <name> elements and their bounding box coordinates
#             objects = []
#             for obj in root.findall(".//object"):
#                 name_element = obj.find("name")
#                 bndbox = obj.find("bndbox")
#                 if name_element is not None and bndbox is not None:
#                     xmin = int(bndbox.find("xmin").text)
#                     ymin = int(bndbox.find("ymin").text)
#                     objects.append({
#                         "name": name_element.text,
#                         "xmin": xmin,
#                         "ymin": ymin
#                     })

#             # Sort objects by (ymin, xmin) to order them from top-left to bottom-right
#             objects.sort(key=lambda x: (x["ymin"], x["xmin"]))

#             # Extract the sorted names
#             names = [obj["name"] for obj in objects]

#             # Get the corresponding image file name (assuming it has the same base name as the XML)
#             image_base_name = os.path.splitext(file_name)[0]
#             image_extensions = [".jpg", ".png"]  # Add more extensions if needed
#             image_file = None

#             # Check for the existence of the image file with supported extensions
#             for ext in image_extensions:
#                 image_path = os.path.join(input_folder, f"{image_base_name}{ext}")
#                 if os.path.exists(image_path):
#                     image_file = image_path
#                     break

#             if image_file:
#                 # Rename the XML file
#                 new_xml_name = f"{file_counter}.xml"
#                 new_xml_path = os.path.join(input_folder, new_xml_name)
#                 os.rename(file_path, new_xml_path)

#                 # Rename the image file
#                 new_image_name = f"{file_counter}{os.path.splitext(image_file)[1]}"
#                 new_image_path = os.path.join(input_folder, new_image_name)
#                 os.rename(image_file, new_image_path)

#                 # Write the new image name (without extension) and sorted names to the CSV
#                 csv_writer.writerow([os.path.splitext(new_image_name)[0], " ".join(names)])

#                 # Increment the counter for the next file
#                 file_counter += 1

# print(f"CSV file has been created at {output_csv}")

CSV file has been created at /content/drive/MyDrive/Colab Notebooks/Dataset/output.csv


In [14]:
import os
import xml.etree.ElementTree as ET
import csv

# Specify the folder containing the XML files
input_folder = "/content/drive/MyDrive/Colab Notebooks/Dataset/Character Recognition/test/"
output_csv = "/content/drive/MyDrive/Colab Notebooks/Dataset/output.csv"

# Define character and number categories
char_order = {"ka", "kha", "ga", "gha", "ca", "ja", "ta", "da", "na", "ba", "va", "ha", "metro", "dhaka", "khulna", "chattogram", "jashore"}
num_order = {"0", "1", "2", "3", "4", "5", "6", "7", "8", "9"}

# Prepare the CSV file for writing
with open(output_csv, mode="w", newline="", encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header row
    csv_writer.writerow(["file_name", "txt"])

    # Iterate through all files in the folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".xml"):
            # Parse the XML file
            file_path = os.path.join(input_folder, file_name)
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Extract <name> elements and their corresponding xmin values
            elements = []
            for obj in root.findall(".//object"):
                name_element = obj.find("name")
                xmin_element = obj.find("bndbox/xmin")

                if name_element is not None and xmin_element is not None:
                    name = name_element.text
                    xmin = int(xmin_element.text)
                    elements.append((xmin, name))

            # Sort elements by xmin
            elements.sort()

            # Separate characters and numbers
            char_list = [name for _, name in elements if name in char_order]
            num_list = [name for _, name in elements if name in num_order]

            # Combine sorted results
            sorted_names = char_list + num_list

            # Write to CSV
            file_base_name = os.path.splitext(file_name)[0]
            csv_writer.writerow([file_base_name, " ".join(sorted_names)])

print(f"CSV file has been created at {output_csv}")


CSV file has been created at /content/drive/MyDrive/Colab Notebooks/Dataset/output.csv


In [4]:
import pandas as pd

def convert_to_bengali(input_text, conversion_dict):
    words = input_text.split()
    converted_words = [conversion_dict.get(word, word) for word in words]
    return ' '.join(converted_words)

def main():
    # File path to the Excel file
    input_file = "/content/drive/MyDrive/Colab Notebooks/Dataset/output.csv"
    output_file = "/content/drive/MyDrive/Colab Notebooks/Dataset/output1.xlsx"

    # Define the mapping dictionary
    conversion_dict = {
        "0": "০", "1": "১", "2": "২", "3": "৩", "4": "৪",
        "5": "৫", "6": "৬", "7": "৭", "8": "৮", "9": "৯",
        "ka": "ক", "kha": "খ", "ga": "গ", "gha": "ঘ", "ca": "চ",
        "ja": "জ", "ta": "ট", "da": "ড", "na": "ন", "ba": "ব",
        "va": "ভ", "ha": "হ", "metro": "মেট্রো", "dhaka": "ঢাকা",
        "khulna": "খুলনা", "chattogram": "চট্টগ্রাম", "jashore": "যোশর"
    }

    # Read the Excel file
    df = pd.read_csv(input_file)

    # Check if the second column exists
    if df.shape[1] < 2:
        print("Error: The file does not have a second column.")
        return

    # Convert the second column
    df["Converted Column"] = df.iloc[:, 1].apply(lambda x: convert_to_bengali(str(x), conversion_dict))

    # Save the output to a new Excel file
    df.to_excel(output_file, index=False)
    print(f"Conversion completed. Output saved to {output_file}")

if __name__ == "__main__":
    main()

Conversion completed. Output saved to /content/drive/MyDrive/Colab Notebooks/Dataset/output1.xlsx


In [12]:
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET

# Path to the XML file and corresponding image
xml_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/Character Recognition/test/79_93_1_jpg.rf.f9b4b91ce905bdc995ff9f8a2312001a.xml"
image_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/Character Recognition/test/79_93_1_jpg.rf.f9b4b91ce905bdc995ff9f8a2312001a.jpg"

# Parse the XML file
tree = ET.parse(xml_path)
root = tree.getroot()

# Load the image
image = Image.open(image_path)
draw = ImageDraw.Draw(image)

# Iterate through all objects in the XML
for obj in root.findall("object"):
    name = obj.find("name").text
    bndbox = obj.find("bndbox")
    xmin = int(bndbox.find("xmin").text)
    ymin = int(bndbox.find("ymin").text)
    xmax = int(bndbox.find("xmax").text)
    ymax = int(bndbox.find("ymax").text)

    # Draw the bounding box
    draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=2)

    # Add the name as a label
    draw.text((xmin, ymin - 10), name, fill="red")

# Display the image
image.show()

# Save the image with bounding boxes (optional)
output_image_path = "/content/output_image.jpg"
image.save(output_image_path)
print(f"Image with bounding boxes saved to {output_image_path}")

Image with bounding boxes saved to /content/output_image.jpg
